# Importações e Bibliotecas

Nesta seção, importamos as bibliotecas necessárias do `keras` para manipular o dataset MNIST:

- `mnist` de `keras.datasets`: para carregar o dataset MNIST.
- `layers` e `models` de `keras`: para a construção de modelos de aprendizado profundo.
- `to_categorical` de `keras.utils`: para converter rótulos numéricos em vetores categóricos, que são necessários para a classificação multi-classe.


In [1]:
from keras.datasets import mnist
from keras import layers
from keras import models
from keras.utils import to_categorical


# Carregamento e Pré-processamento dos Dados


1. **Carregamento dos Dados**: Utilizamos a função `load_data()` do `mnist` para separar os dados em conjuntos de treinamento e teste. O conjunto de treinamento contém 60.000 imagens, enquanto o conjunto de teste possui 10.000 imagens, cada uma com dimensões de 28x28 pixels.

2. **Redimensionamento e Normalização**:
   - As imagens são redimensionadas para incluir um único canal de cor, ajustando sua forma para (60000, 28, 28, 1) para o conjunto de treinamento e para (10000, 28, 28, 1) para o conjunto de teste.
   - Convertendo as imagens para o tipo `float32` e normalizando os valores dos pixels por 255, facilitamos o treinamento do modelo ao utilizar valores de entrada entre 0 e 1.


In [2]:
# Carregamento dos dados
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32')/255


# Transformando os Rótulos em Categorias

Para que os rótulos dos dados sejam compatíveis com o modelo de classificação que será construído, é necessário convertê-los de formatos numéricos simples para vetores categóricos. Este processo é fundamental para a classificação multi-classe em redes neurais, onde cada categoria precisa ser representada por um vetor binário. Utilizamos a função `to_categorical` de `keras.utils` para essa transformação:

- `train_labels = to_categorical(train_labels)`: Converte os rótulos de treinamento em vetores categóricos.
- `test_labels = to_categorical(test_labels)`: Converte os rótulos de teste em vetores categóricos.

Esse passo garante que a rede neural possa entender corretamente os rótulos durante o treinamento e a avaliação do modelo.


In [3]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


# Rede Convolucional Simples

Primeiro teste simples sobre redes convolucionais. A arquitetura da rede inclui uma camada convolucional, uma de flattening e densas.

## Arquitetura da Rede


1. **Primeira Camada Convolucional**: Aplicamos 32 filtros de tamanho 3x3 com função de ativação ReLU.
2. **Flatten**: Convertendo a saída da camada convolucional em um vetor 1D.
3. **Primeira Camada Densa**: Contém 250 neurônios com função de ativação sigmoide.
4. **Camada de Saída**: Contém 10 neurônios (um para cada classe) com função de ativação softmax.

In [5]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.Flatten())
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 250)            │     5,408,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,411,080 (20.64 MB)

 Trainable params: 5,411,080 (20.64 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [7]:
model.fit(train_images, train_labels, epochs=5, batch_size = 64)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 33s 34ms/step - accuracy: 0.9029 - loss: 0.3146
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 32s 34ms/step - accuracy: 0.9852 - loss: 0.0514
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.9929 - loss: 0.0256
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 32s 34ms/step - accuracy: 0.9960 - loss: 0.0132
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.9978 - loss: 0.0074


In [8]:
model.evaluate(test_images, test_labels)[1]

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9826 - loss: 0.0664


0.9865999817848206

In [9]:
def treinar(modelo):
    modelo.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

    modelo.fit(train_images, train_labels, epochs=5, batch_size = 64)

    return modelo

In [10]:
def stats(modelo):
    return modelo.evaluate(test_images, test_labels)[1]  # Index 1 for accuracy


In [11]:
#def treinamento(primeiro_layer,epocas,ultimo_layer):
 # model = Model([inpx], ultimo_layer)
  #model.compile(optimizer=keras.optimizers.Adadelta(),
   #     loss=keras.losses.categorical_crossentropy,
    #    metrics=['accuracy'])

 # model.fit(x_train, y_train, epochs=epocas, batch_size=500)

In [12]:
#def stats(x):
 # score = x.evaluate(x_test, y_test, verbose=0)
  #print('loss=', score[0])
  #print('accuracy=', score[1])

# Testes com diferentes tamanhos de filtro

### Teste 1 (5x5)

#### Arquitetura da Rede


1. **Primeira Camada Convolucional**: Aplicamos 32 filtros de tamanho 5x5 com função de ativação ReLU.
2. **Flatten**: Convertendo a saída da camada convolucional em um vetor 1D.
3. **Primeira Camada Densa**: Contém 250 neurônios com função de ativação sigmoide.
4. **Camada de Saída**: Contém 10 neurônios (um para cada classe) com função de ativação softmax.

In [13]:
model5x5 = models.Sequential()
model5x5.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(28, 28, 1)))
model5x5.add(layers.Flatten())
model5x5.add(layers.Dense(250, activation='relu'))
model5x5.add(layers.Dense(10, activation='softmax'))

model5x5.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 24, 24, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 250)            │     4,608,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,611,592 (17.59 MB)

 Trainable params: 4,611,592 (17.59 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
treinar(model5x5)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 26s 28ms/step - accuracy: 0.9063 - loss: 0.3017
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 26s 28ms/step - accuracy: 0.9861 - loss: 0.0454
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 26s 28ms/step - accuracy: 0.9934 - loss: 0.0215
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 26s 28ms/step - accuracy: 0.9962 - loss: 0.0131
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 25s 27ms/step - accuracy: 0.9981 - loss: 0.0064


<Sequential name=sequential_2, built=True>

In [15]:
stats(model5x5)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9842 - loss: 0.0544


0.9868999719619751

# Teste 7x7

Este teste explora uma variação da arquitetura de rede convolucional utilizando um filtro de maior dimensão (7x7) na camada convolucional para investigar o impacto no aprendizado de características mais abrangentes das imagens.

#### Arquitetura da Rede

1. **Primeira Camada Convolucional**: Aplicamos 32 filtros de tamanho 7x7 com função de ativação ReLU. A escolha de um filtro maior pode ajudar a capturar padrões visuais mais complexos desde a primeira camada.
2. **Flatten**: A saída da camada convolucional é convertida em um vetor unidimensional (1D) para permitir a conexão com camadas densas subsequentes.
3. **Primeira Camada Densa**: Contém 250 neurônios com função de ativação ReLU.
4. **Camada de Saída**: Contém 10 neurônios, um para cada classe do conjunto de dados MNIST, com função de ativação softmax para a classificação multi-classe.

#### Sumário do Modelo

O sumário do modelo fornece uma visão detalhada das camadas, mostrando a forma da saída e o número de parâmetros treináveis em cada camada, ajudando a entender a complexidade e a capacidade do modelo.

- **Modelo**: "sequential_3"
- **Total de Parâmetros**: 3.876.360, dos quais todos são parâmetros treináveis, indicando a capacidade de aprendizado do modelo.
- A função `model.summary()` é utilizada para visualizar esta informação estrutural e paramétrica do modelo.




In [16]:
model7x7 = models.Sequential()
model7x7.add(layers.Conv2D(32, (7, 7), activation='relu', input_shape=(28, 28, 1)))
model7x7.add(layers.Flatten())
model7x7.add(layers.Dense(250, activation='relu'))
model7x7.add(layers.Dense(10, activation='softmax'))

model7x7.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 22, 22, 32)     │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 15488)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 250)            │     3,872,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,876,360 (14.79 MB)

 Trainable params: 3,876,360 (14.79 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
treinar(model7x7)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.9197 - loss: 0.2550
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.9881 - loss: 0.0387
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.9933 - loss: 0.0219
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - accuracy: 0.9959 - loss: 0.0130
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - accuracy: 0.9975 - loss: 0.0085


<Sequential name=sequential_3, built=True>

In [18]:
stats(model7x7)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9872 - loss: 0.0422


0.9890999794006348


# Teste 25x25

Este teste explora uma arquitetura de rede convolucional com um filtro de tamanho excepcionalmente grande (25x25), visando investigar sua eficácia na captura de características visuais em uma escala mais ampla das imagens.

#### Arquitetura da Rede

1. **Primeira Camada Convolucional**: Implementamos 32 filtros de tamanho 25x25 com função de ativação ReLU. Este tamanho de filtro grande é utilizado para capturar padrões visuais extensos, potencialmente abrangendo quase toda a área da imagem de entrada.
2. **Flatten**: Converte a saída da camada convolucional em um vetor unidimensional (1D), preparando-a para a conexão com as camadas densas.
3. **Primeira Camada Densa**: Contém 250 neurônios com função de ativação ReLU.
4. **Camada de Saída**: Contém 10 neurônios, um para cada classe do conjunto de dados MNIST, equipada com função de ativação softmax para a classificação multi-classe.

#### Sumário do Modelo

O sumário do modelo oferece uma visão detalhada da arquitetura e parâmetros:

- **Modelo**: "sequential_4"
- **Total de Parâmetros**: 1.507.692, todos treináveis, indicando a grande capacidade e complexidade do modelo devido ao tamanho do filtro convolucional utilizado.
- A função `model.summary()` é empregada para exibir as informações estruturais e paramétricas, facilitando a análise da arquitetura do modelo.



In [19]:
model25x25 = models.Sequential()
model25x25.add(layers.Conv2D(32, (25,25), activation='relu', input_shape=(28, 28, 1)))
model25x25.add(layers.Flatten())
model25x25.add(layers.Dense(250, activation='relu'))
model25x25.add(layers.Dense(10, activation='softmax'))

model25x25.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 4, 4, 32)       │        20,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 250)            │       128,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 150,792 (589.03 KB)

 Trainable params: 150,792 (589.03 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
treinar(model25x25)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8657 - loss: 0.4331
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9691 - loss: 0.0985
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9778 - loss: 0.0694
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9838 - loss: 0.0537
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9865 - loss: 0.0442


<Sequential name=sequential_4, built=True>

In [21]:
stats(model25x25)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.9754 - loss: 0.0927


0.9794999957084656

## Resultados parciais foram inconclusivos com apenas uma camada de convolução

- 3x3: treinamento extremamente lento, muitos parâmetros e bons resultados.
- 5x5: treinamento razoavelmente lento, muitos parâmetros e bons resultados.
- 7x7: treinamento em tempo aceitável, quantidade menor de parâmetros e melhores resultados ( lembrando que usamos apenas UMA camada ).
- 25x25: treinamento muito rápido, quantidade pequena de parâmetros e piores resultados, o que faz sentido pois são filtros 25x25 para imagens 28x28.

Esses achados sugerem que a seleção do tamanho do filtro na camada convolucional deve considerar não apenas a eficiência computacional, mas também a capacidade do modelo de aprender características essenciais das imagens, adequadas ao tamanho e detalhe dos dados de entrada.

#

# Múltiplas Camadas de Convolução

Este teste avalia o impacto de utilizar múltiplas camadas convolucionais em sequência, cada uma com filtros de tamanho 3x3. A utilização de múltiplas camadas convolucionais é uma prática comum em redes neurais profundas para extrair hierarquias de características mais complexas das imagens.

#### Arquitetura da Rede

1. **Camadas Convolucionais**:
   - **Primeira Camada Convolucional**: 32 filtros de 3x3, ativação ReLU. Esta camada busca captar as características primárias da imagem.
   - **Segunda Camada Convolucional**: 32 filtros de 3x3, ativação ReLU. Ao aplicar uma segunda camada convolucional, aprofundamos a análise das características abstratas identificadas inicialmente.
   - **Terceira Camada Convolucional**: 32 filtros de 3x3, ativação ReLU. A terceira camada permite uma extração ainda mais refinada e detalhada das características.
2. **Flatten**: Transforma a saída 3D das camadas convolucionais em um vetor 1D para permitir a conexão com camadas densas.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: 250 neurônios, ativação ReLU.
   - **Camada de Saída**: 10 neurônios, um para cada classe, com ativação softmax para a classificação multi-classe.

#### Sumário do Modelo

A função `model.summary()` é utilizada para exibir o sumário do modelo, que inclui detalhes sobre as camadas, formas de saída e quantidade de parâmetros. Este sumário ajuda a visualizar a estrutura e a complexidade da rede:




In [22]:
model_camadas_3 = models.Sequential()
model_camadas_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas_3.add(layers.Flatten())
model_camadas_3.add(layers.Dense(250, activation='relu'))
model_camadas_3.add(layers.Dense(10, activation='softmax'))

model_camadas_3.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 22, 22, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 15488)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 250)            │     3,872,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,893,576 (14.85 MB)

 Trainable params: 3,893,576 (14.85 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
treinar(model_camadas_3)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 32ms/step - accuracy: 0.9104 - loss: 0.2895
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.9888 - loss: 0.0358
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 30s 32ms/step - accuracy: 0.9940 - loss: 0.0184
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 32s 34ms/step - accuracy: 0.9966 - loss: 0.0111
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.9978 - loss: 0.0080


<Sequential name=sequential_5, built=True>

In [24]:
stats(model_camadas_3)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9842 - loss: 0.0742


0.988099992275238

# Múltiplas Camadas de Convolução com Filtros 8x8

Neste teste, exploramos o impacto de utilizar filtros de maior dimensão (8x8) em três camadas convolucionais consecutivas. O objetivo é investigar como filtros maiores influenciam a capacidade da rede de extrair características visuais em escalas maiores, o que pode ser vantajoso para capturar padrões mais globais nas imagens.

#### Arquitetura da Rede

1. **Camadas Convolucionais**:
   - **Primeira Camada Convolucional**: 32 filtros de 8x8, ativação ReLU. Esta camada inicia o processo de extração de características, aplicando filtros maiores para uma visão mais abrangente das imagens.
   - **Segunda Camada Convolucional**: 32 filtros de 8x8, ativação ReLU. Aprofunda a análise das características identificadas pela primeira camada, reforçando a detecção de padrões mais amplos.
   - **Terceira Camada Convolucional**: 32 filtros de 8x8, ativação ReLU. Esta última camada convolucional maximiza a extração de detalhes antes da transição para camadas densas.
2. **Flatten**: Converte a saída das camadas convolucionais em um vetor 1D, preparando a rede para as etapas de classificação.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: 250 neurônios, ativação ReLU.
   - **Camada de Saída**: 10 neurônios, correspondendo às dez classes do dataset, com ativação softmax para classificação multi-classe.




In [25]:
model_camadas_8x8 = models.Sequential()
model_camadas_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1)))
model_camadas_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1)))
model_camadas_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1)))
model_camadas_8x8.add(layers.Flatten())
model_camadas_8x8.add(layers.Dense(250, activation='relu'))
model_camadas_8x8.add(layers.Dense(10, activation='softmax'))

model_camadas_8x8.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 21, 21, 32)     │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 32)     │        65,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 7, 7, 32)       │        65,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 1568)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 250)            │       392,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 527,976 (2.01 MB)

 Trainable params: 527,976 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
treinar(model_camadas_8x8)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.8728 - loss: 0.3855
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9848 - loss: 0.0531
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9897 - loss: 0.0374
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9920 - loss: 0.0292
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9938 - loss: 0.0232


<Sequential name=sequential_6, built=True>

In [27]:
stats(model_camadas_8x8)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9866 - loss: 0.0458


0.9901000261306763

# Testando a Utilização de Padding no Teste 8x8

Este experimento explora a implementação de 'padding' do tipo 'same' em uma arquitetura de rede convolucional com filtros 8x8. O 'padding' 'same' permite que a rede mantenha a dimensão espacial da entrada nas camadas convolucionais, possibilitando uma análise mais profunda sem redução do tamanho das características de entrada ao longo das camadas.

#### Arquitetura da Rede

1. **Camadas Convolucionais**:
   - **Primeira Camada Convolucional**: 32 filtros de 8x8 com ativação ReLU e padding 'same'. Esta configuração mantém a dimensão da entrada (28x28), focando na extração detalhada de características sem perda espacial.
   - **Segunda Camada Convolucional**: Repete a configuração com 32 filtros de 8x8, ativação ReLU e padding 'same', proporcionando continuidade na análise de características com a mesma dimensionalidade.
   - **Terceira Camada Convolucional**: Semelhante às anteriores, com 32 filtros de 8x8, ativação ReLU e padding 'same', maximizando a profundidade da análise característica sem redução de tamanho.
2. **Flatten**: A saída 3D das camadas convolucionais é transformada em um vetor 1D, preparando a rede para as camadas densas.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: 250 neurônios com função de ativação ReLU.
   - **Camada de Saída**: 10 neurônios, correspondendo às dez classes do dataset, com ativação softmax para classificação.



In [28]:
model_camadas_padding_8x8 = models.Sequential()
model_camadas_padding_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8x8.add(layers.Conv2D(32, (8,8), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8x8.add(layers.Flatten())
model_camadas_padding_8x8.add(layers.Dense(250, activation='relu'))
model_camadas_padding_8x8.add(layers.Dense(10, activation='softmax'))

model_camadas_padding_8x8.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_13 (Conv2D)              │ (None, 28, 28, 32)     │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 28, 28, 32)     │        65,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 28, 28, 32)     │        65,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 250)            │     6,272,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,407,976 (24.44 MB)

 Trainable params: 6,407,976 (24.44 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
treinar(model_camadas_padding_8x8)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 93s 98ms/step - accuracy: 0.8895 - loss: 0.3298
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 94s 101ms/step - accuracy: 0.9885 - loss: 0.0385
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 93s 99ms/step - accuracy: 0.9925 - loss: 0.0249
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 91s 97ms/step - accuracy: 0.9945 - loss: 0.0195
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 95s 101ms/step - accuracy: 0.9954 - loss: 0.0155


<Sequential name=sequential_7, built=True>

# Resultados sobre o Uso de Padding

A utilização de padding 'same' no modelo com filtros 8x8 foi analisada para entender o impacto no número de parâmetros e na performance do modelo. O padding 'same' mantém as dimensões de entrada ao longo das camadas, o que evita a redução do tamanho do output das camadas convolucionais, mas também influencia a eficácia geral da rede.

### Análise Comparativa de Performance

- **Sem Padding**: A performance do modelo sem padding mostrou uma acurácia de 0.9919000267982483. Este resultado indica alta eficiência, provavelmente devido à redução gradual das dimensões, que pode ajudar na focalização de características importantes.
- **Com Padding**: Com a aplicação de padding 'same', a acurácia foi ligeiramente menor, com um valor de 0.9914000034332275. A pequena diminuição na acurácia sugere que, enquanto o modelo com padding mantém mais informações espaciais, ele pode não ser tão efetivo em extrair e destacar características úteis para classificação em alguns casos.

#### Considerações Finais

O aumento no número de parâmetros devido ao uso de padding 'same' não se traduziu necessariamente em uma melhoria da performance do modelo. O modelo sem padding ainda apresentou resultados ligeiramente melhores, o que indica que, para este projeto específico, a redução dimensional pode ser benéfica na concentração de características significativas para a classificação. Porém, é importante considerar que o uso de padding pode ser mais apropriado em contextos onde a preservação das informações espaciais completas é crucial.



In [30]:
stats(model_camadas_padding_8x8)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9897 - loss: 0.0544


0.9901999831199646

### Ainda mais Camadas Convolucionais 3x3

Este modelo explora o impacto de empilhar múltiplas camadas convolucionais, todas com filtros de tamanho 3x3, para capturar uma hierarquia complexa de características visuais. A ideia é analisar como camadas adicionais influenciam a capacidade do modelo de entender detalhes em diferentes níveis de abstração das imagens.

#### Arquitetura da Rede

1. **Camadas Convolucionais**: Cada uma das oito camadas utiliza 32 filtros de tamanho 3x3 com função de ativação ReLU, configuradas para extrair características progressivamente mais abstratas à medida que a profundidade aumenta:
   - As camadas são aplicadas sucessivamente, cada uma recebendo a saída da camada anterior como entrada, permitindo que o modelo construa uma compreensão detalhada das características em várias escalas espaciais.
2. **Flatten**: Converte a saída tridimensional das camadas convolucionais em um vetor unidimensional, permitindo que os dados sejam processados por camadas densas subsequentes.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: 250 neurônios com ativação ReLU, para realizar uma transformação não-linear e combinar as características aprendidas.
   - **Camada de Saída**: 10 neurônios, um para cada classe do dataset, utilizando a função de ativação softmax para realizar a classificação final.



In [31]:

model_camadas8_3 = models.Sequential()
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)))
model_camadas8_3.add(layers.Flatten())
model_camadas8_3.add(layers.Dense(250, activation='relu'))
model_camadas8_3.add(layers.Dense(10, activation='softmax'))

model_camadas8_3.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 22, 22, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 20, 20, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 18, 18, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 14, 14, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 12, 12, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 250)            │     1,152,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,816 (4.65 MB)

 Trainable params: 1,219,816 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
treinar(model_camadas8_3)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 29ms/step - accuracy: 0.8644 - loss: 0.4073
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 29ms/step - accuracy: 0.9836 - loss: 0.0534
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 30ms/step - accuracy: 0.9894 - loss: 0.0361
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 29ms/step - accuracy: 0.9913 - loss: 0.0280
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 28s 30ms/step - accuracy: 0.9935 - loss: 0.0227


<Sequential name=sequential_8, built=True>

In [33]:
stats(model_camadas8_3)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9854 - loss: 0.0498


0.9884999990463257

### Utilizando a Mesma Rede para Outro Teste com Padding

Neste novo teste, utilizamos a mesma configuração de rede anterior, que empilha várias camadas convolucionais com padding do tipo 'same'. O objetivo é verificar se o uso consistente de padding 'same' ao longo de múltiplas camadas pode melhorar a capacidade de extração de características sem perder informações de borda.

#### Arquitetura da Rede

1. **Camadas Convolucionais**:
   - Oito camadas convolucionais são configuradas com 32 filtros de tamanho 3x3 cada, utilizando a função de ativação ReLU e padding 'same'. Esta configuração visa manter a dimensão espacial das entradas ao longo das camadas, permitindo uma análise detalhada das características sem redução de tamanho.
   - O uso repetido de padding 'same' é projetado para preservar as dimensões espaciais das imagens de entrada, permitindo que as camadas profundas continuem a processar a totalidade da informação visual sem perda de contexto ou bordas.
2. **Flatten**: A saída das camadas convolucionais é transformada em um vetor unidimensional, preparando os dados para processamento pelas camadas densas.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: Contém 250 neurônios com ativação ReLU, integrando as características extraídas para formar representações mais abstratas.
   - **Camada de Saída**: Composta por 10 neurônios, cada um representando uma classe do dataset, e utilizando a função de ativação softmax para a classificação final.


Mais uma vez resultados não tão interessantes utilizando padding


In [34]:
model_camadas_padding_8_3 = models.Sequential()
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model_camadas_padding_8_3.add(layers.Flatten())
model_camadas_padding_8_3.add(layers.Dense(250, activation='relu'))
model_camadas_padding_8_3.add(layers.Dense(10, activation='softmax'))

model_camadas_padding_8_3.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 250)            │     6,272,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,339,816 (24.18 MB)

 Trainable params: 6,339,816 (24.18 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
treinar(model_camadas_padding_8_3)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 79s 83ms/step - accuracy: 0.8739 - loss: 0.3905
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 77s 82ms/step - accuracy: 0.9869 - loss: 0.0431
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 77s 82ms/step - accuracy: 0.9923 - loss: 0.0252
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 76s 81ms/step - accuracy: 0.9946 - loss: 0.0163
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 79s 84ms/step - accuracy: 0.9966 - loss: 0.0123


<Sequential name=sequential_9, built=True>

In [36]:
stats(model_camadas_padding_8_3)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9885 - loss: 0.0404


0.9904000163078308

### Testando a utilização de stride

Neste experimento, investigamos o impacto de aplicar um stride maior (2,2) em todas as camadas convolucionais. O stride, que se refere ao deslocamento do filtro ao aplicar a convolução, afeta diretamente o tamanho da saída e a quantidade de operações computacionais, podendo influenciar significativamente tanto o tempo de treinamento quanto a performance do modelo.

#### Arquitetura da Rede

1. **Camadas Convolucionais**: A rede é composta por várias camadas convolucionais que utilizam um stride de (2,2) para reduzir as dimensões da saída a cada camada, minimizando assim a complexidade computacional:
   - **Primeira Camada Convolucional**: 32 filtros de 3x3, ativação ReLU, stride de (2,2). Esta configuração ajuda a captar características essenciais enquanto reduz a resolução da entrada.
   - **Camadas Convencionais Subsequentes**: Repetem a mesma configuração de filtros, ativação e stride, promovendo uma redução progressiva na dimensionalidade das características, permitindo que a rede processe informações de forma mais eficiente.
2. **Flatten**: Transforma a saída 3D das camadas convolucionais em um vetor unidimensional, necessário para as transições para as camadas densas.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: Composta por 250 neurônios com função de ativação ReLU, para a integração das características.
   - **Camada de Saída**: Contém 10 neurônios, um para cada classe do dataset, com função de ativação softmax, adequada para realizar a classificação.




In [37]:
model_camadas_stride_3x3 = models.Sequential()
model_camadas_stride_3x3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1), strides=(2,2)))
model_camadas_stride_3x3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1), strides=(2,2)))
model_camadas_stride_3x3.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1), strides=(2,2)))
model_camadas_stride_3x3.add(layers.Flatten())
model_camadas_stride_3x3.add(layers.Dense(250, activation='relu'))
model_camadas_stride_3x3.add(layers.Dense(10, activation='softmax'))
model_camadas_stride_3x3.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_32 (Conv2D)              │ (None, 13, 13, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 6, 6, 32)       │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 2, 2, 32)       │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 250)            │        32,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,576 (209.28 KB)

 Trainable params: 53,576 (209.28 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
treinar(model_camadas_stride_3x3)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8553 - loss: 0.4968
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9720 - loss: 0.0900
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9819 - loss: 0.0565
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9863 - loss: 0.0459
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9888 - loss: 0.0356


<Sequential name=sequential_10, built=True>

In [39]:
stats(model_camadas_stride_3x3)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - accuracy: 0.9754 - loss: 0.0842


0.978600025177002

# Testes variando o Pooling

Estamos explorando variações nos métodos de pooling, incluindo Max Pooling e Average Pooling, para entender como cada abordagem afeta o desempenho e a eficiência do modelo em processar características espaciais das imagens.

## Max Pooling

Utilizamos o Max Pooling na arquitetura da rede para reduzir as dimensões espaciais das saídas das camadas convolucionais, o que ajuda a diminuir a quantidade de parâmetros e a carga computacional, ao mesmo tempo em que preserva as características mais importantes identificadas pelos filtros.

#### Arquitetura da Rede com Max Pooling

1. **Camadas Convolucionais e Pooling**:
   - **Primeira Camada Convolucional**: 32 filtros de tamanho 3x3 com ativação ReLU, seguida por uma camada de Max Pooling com tamanho de pool 3x3.
   - **Segunda Camada Convolucional**: 64 filtros de tamanho 3x3 com ativação ReLU, também seguida por Max Pooling com tamanho de pool 3x3. Esta configuração permite uma extração eficiente de características ao reduzir progressivamente o tamanho da saída.
2. **Flatten**: Converte a saída das camadas de pooling em um vetor unidimensional, facilitando a transição para as camadas densas.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: 128 neurônios com ativação ReLU.
   - **Dropout**: Aplicação de dropout com taxa de 0.5 para reduzir o overfitting durante o treinamento.
   - **Camada de Saída**: 10 neurônios, um para cada classe do dataset, com ativação softmax para a classificação final.



In [40]:
model_max_pooling = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(3, 3)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_max_pooling.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_35 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 6, 6, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,002 (207.04 KB)

 Trainable params: 53,002 (207.04 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
treinar(model_max_pooling)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7851 - loss: 0.6842
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9652 - loss: 0.1135
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9776 - loss: 0.0731
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9819 - loss: 0.0569
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9849 - loss: 0.0492


<Sequential name=sequential_11, built=True>

In [42]:
stats(model_max_pooling)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9855 - loss: 0.0393


0.9883999824523926

## Average Pooling

A técnica de Average Pooling é explorada neste modelo para comparar seus efeitos com os do Max Pooling. O Average Pooling calcula a média dos valores em cada janela de pooling, o que pode contribuir para uma representação mais suavizada das características em comparação ao Max Pooling, que destaca as características mais proeminentes.

#### Arquitetura da Rede com Average Pooling

1. **Camadas Convolucionais e Pooling**:
   - **Primeira Camada Convolucional**: 32 filtros de tamanho 3x3 com ativação ReLU, seguida por uma camada de Average Pooling com tamanho de pool 3x3.
   - **Segunda Camada Convolucional**: 64 filtros de tamanho 3x3 com ativação ReLU, também seguida por uma camada de Average Pooling com tamanho de pool 3x3. Esta configuração visa reduzir as dimensões espaciais da saída enquanto preserva uma representação média das características detectadas.
2. **Flatten**: Transforma a saída das camadas de pooling em um vetor unidimensional, permitindo a transição para as camadas densas.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: 128 neurônios com ativação ReLU.
   - **Dropout**: Implementação de uma camada de Dropout com taxa de 0.5 para minimizar o risco de overfitting durante o treinamento.
   - **Camada de Saída**: 10 neurônios, um para cada classe do dataset, com função de ativação softmax para a classificação final.




In [43]:
model_avg_pooling = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.AveragePooling2D(pool_size=(3, 3)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.AveragePooling2D(pool_size=(3, 3)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_avg_pooling.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_37 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 8, 8, 32)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 6, 6, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 2, 2, 64)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,002 (207.04 KB)

 Trainable params: 53,002 (207.04 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
treinar(model_avg_pooling)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6940 - loss: 0.9495
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9245 - loss: 0.2463
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9468 - loss: 0.1740
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9572 - loss: 0.1383
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9659 - loss: 0.1130


<Sequential name=sequential_12, built=True>

In [45]:
stats(model_avg_pooling)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9799 - loss: 0.0697


0.9825000166893005

# Testes com diferentes valores para o Max Pooling

Dada a superioridade observada nos testes anteriores com o Max Pooling em comparação ao Average Pooling, neste segmento focamos em variar o tamanho do pooling para avaliar como diferentes configurações de Max Pooling afetam a performance e a capacidade de generalização do modelo.

## Max Pooling 2x2

Optamos por testar o Max Pooling com um tamanho de pool de 2x2, uma configuração comum que oferece um bom equilíbrio entre redução dimensional e preservação de características importantes.

#### Arquitetura da Rede com Max Pooling 2x2

1. **Camadas Convolucionais e Pooling**:
   - **Primeira Camada Convolucional**: 32 filtros de tamanho 3x3 com ativação ReLU. Após esta camada, aplicamos Max Pooling com pool size de 2x2 para começar a redução das dimensões espaciais da imagem.
   - **Segunda Camada Convolucional**: 64 filtros de tamanho 3x3 com ativação ReLU. Segue-se outra camada de Max Pooling com pool size de 2x2, continuando a compactação das características extraídas enquanto mantém as mais essenciais.
   - Repetição de camadas convolucionais seguidas por Max Pooling para maximizar a eficiência na captura de características.
2. **Flatten**: Transforma as saídas das camadas de pooling em um vetor unidimensional para facilitar o processamento pelas camadas densas.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: 128 neurônios com ativação ReLU.
   - **Dropout**: Implementação de dropout com taxa de 0.5 para reduzir o risco de overfitting.
   - **Camada de Saída**: 10 neurônios, cada um representando uma classe do dataset, com ativação softmax para a classificação final.




In [46]:
model_max_pooling2 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_max_pooling2.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_39 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [47]:
treinar(model_max_pooling2)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.8561 - loss: 0.4547
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9759 - loss: 0.0798
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9820 - loss: 0.0588
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9868 - loss: 0.0454
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9875 - loss: 0.0402


<Sequential name=sequential_13, built=True>

In [48]:
stats(model_max_pooling2)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9879 - loss: 0.0351


0.9908000230789185

## Max Pooling 5x5

Neste segmento do experimento, avaliamos o impacto do uso de Max Pooling com um tamanho de pool de 5x5, que é uma configuração consideravelmente maior do que o tradicionalmente utilizado. Este tipo de pooling pode ser especialmente eficaz para reduzir rapidamente as dimensões da saída, mantendo as características mais dominantes em regiões maiores.

### Arquitetura da Rede com Max Pooling 5x5

1. **Camadas Convolucionais e Pooling**:
   - **Primeira Camada Convolucional**: 32 filtros de tamanho 3x3 com ativação ReLU. Seguida de uma camada de Max Pooling 5x5 para uma redução substancial das dimensões espaciais desde as etapas iniciais.
   - **Segunda Camada Convolucional**: Aplicação de 64 filtros de tamanho 1x1 com ativação ReLU. Este ajuste permite manipular a profundidade dos mapas de características sem alterar suas dimensões espaciais, preparando para uma segunda redução.
   - **Segunda Camada de Max Pooling 5x5**: Aplicada após a segunda convolução para continuar a compactação das características enquanto ainda preserva detalhes cruciais.
2. **Flatten**: Conversão da saída das camadas de pooling em um vetor unidimensional, necessário para a transição para as camadas densas.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: 128 neurônios com ativação ReLU.
   - **Dropout**: Implementação de dropout com uma taxa de 0.5 para mitigar o risco de overfitting.
   - **Camada de Saída**: Composta por 10 neurônios, cada um correspondendo a uma classe do dataset, utilizando ativação softmax para a classificação final.



In [49]:
model_max_pooling5 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(5, 5)),  # Reduz para 5x5
    layers.Conv2D(filters=64, kernel_size=(1, 1), activation='relu'),  # Usa kernel 1x1 para não alterar a dimensão
    layers.MaxPooling2D(pool_size=(5, 5)),  # Reduz para 1x1
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_max_pooling5.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_41 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 5, 5, 64)       │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,042 (47.04 KB)

 Trainable params: 12,042 (47.04 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
treinar(model_max_pooling5)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3350 - loss: 1.8737
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7020 - loss: 0.8979
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7890 - loss: 0.6540
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8254 - loss: 0.5464
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8474 - loss: 0.4888


<Sequential name=sequential_14, built=True>

In [51]:
stats(model_max_pooling5)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - accuracy: 0.8869 - loss: 0.3602


0.9000999927520752

# Testes de Dropout

Neste segmento, examinamos o comportamento de uma rede sem a implementação de dropout para avaliar seu desempenho em termos de treinamento e generalização em comparação a redes com dropout.

## Sem Dropout

A arquitetura da rede sem dropout é configurada para avaliar como a ausência desta técnica afeta a capacidade da rede de evitar o overfitting durante o treinamento.

#### Arquitetura da Rede Sem Dropout

1. **Camadas Convolucionais e Pooling**:
   - **Primeira Camada Convolucional**: 32 filtros de tamanho 3x3 com ativação ReLU, seguida de Max Pooling com tamanho de pool 2x2. Esta camada busca extrair características primárias das imagens.
   - **Segunda Camada Convolucional**: 64 filtros de tamanho 3x3 com ativação ReLU, seguida de outra camada de Max Pooling com tamanho de pool 2x2, para uma redução adicional nas dimensões e destaque das características mais relevantes.
2. **Flatten**: Conversão da saída tridimensional das camadas de pooling em um vetor unidimensional para as transições para as camadas densas.
3. **Camadas Densas**:
   - **Primeira Camada Densa**: 128 neurônios com ativação ReLU, que integra as características aprendidas pelas camadas anteriores.
   - **Camada de Saída**: 10 neurônios, cada um correspondendo a uma classe do dataset, com função de ativação softmax, adequada para a classificação final.

In [52]:
model_no_dropout = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.0),
    layers.Dense(10, activation='softmax')
])
model_no_dropout.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_43 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
treinar(model_no_dropout)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8926 - loss: 0.3465
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9863 - loss: 0.0464
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9908 - loss: 0.0299
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9933 - loss: 0.0206
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9949 - loss: 0.0167


<Sequential name=sequential_15, built=True>

In [54]:
stats(model_no_dropout)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9901 - loss: 0.0286


0.9919000267982483

## Dropout Baixo (0.1)

In [55]:
model_dropout_0_1 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(10, activation='softmax')
])
model_dropout_0_1.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_45 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
treinar(model_dropout_0_1)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8857 - loss: 0.3690
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9840 - loss: 0.0510
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9884 - loss: 0.0358
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9924 - loss: 0.0270
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9940 - loss: 0.0209


<Sequential name=sequential_16, built=True>

In [57]:
stats(model_dropout_0_1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9888 - loss: 0.0365


0.9911999702453613

## Dropout Alto (0.5)

In [58]:
model_dropout_0_5 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model_dropout_0_5.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_47 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
treinar(model_dropout_0_5)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8523 - loss: 0.4598
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9756 - loss: 0.0813
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9820 - loss: 0.0592
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9874 - loss: 0.0445
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9890 - loss: 0.0365


<Sequential name=sequential_17, built=True>

In [60]:
stats(model_dropout_0_5)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9890 - loss: 0.0332


0.991599977016449